## 종만북 chapter 16. 비트마스크
내부적으로 이진수를 사용하는 컴퓨터들은 이진법 관련 연산들을 아주 빨리 할 수 있다.  
이 특성을 이용해 정수의 이진수 표현을 자료 구조로 쓰는 기법을 비트마스크(bitmask)라 부른다.  
  
### 비트마스크의 장점.
1. 빠른 수행시간: 비트마스크 연산은 O(1)에 구현되는 것이 많기 때문에, 적절히 사용할 경우 다른 자료구조를 사용하는 것보다 훨씬 빨리 동작. 
2. 간결한 코드: 다양한 집합 연산들을 반복문 없이 한 줄에 쓸 수 있다.
3. 적은 메모리 사용: 더 많은 데이터를 미리 계산해서 저장해 둘 수 있다. 프로그램도 빨라지고, 일반적으로 캐시 효율도 더 좋다.
4. 연관 배열을 배열로 대체: 불린 값 배열을 키로 갖는 연관 배열 개체 `map<vector<bool>, int>`를 사용할 때, 비트마스크를 써서 같은 정보를 정수 변수로 나타내면 단순한 배열 int[]를 사용해 같은 정보를 나타낼 수 있다. 

#### 용어 정의
**bit**: 이진수의 한 자리를 비트라고 부름. 0 혹은 1의 값을 가짐.  
**most significant bit(최상위 비트)**: 부호 없는 N비트 정수형 변수로 N자리 이진수를 쓸 때 $2^{k-1}$에 해당하는 비트.  
**least significant bit(최하위 비트)**: $2^0$에 해당하는 비트.  
어떤 비트의 위치가 1이면 해당 비트가 "켜져있다"고 하고, 0이면 "꺼져있다"고 한다.  

#### 비트 연산자(bitwise operator)
**bitwise AND**: 입력받은 두 정수를 한 비트씩 비교하면서, 두 정수에 해당 비트가 모두 켜져 있을 때만 결과의 비트를 킨다.  
    
    754와 470의 비트별 AND 연산 결과
    
        1  0  1  1  1  1  0  0  1  0  =  754
    AND 0  1  1  1  0  1  0  1  1  0  =  470
    -----------------------------------------
        0  0  1  1  0  1  0  0  1  0  =  210

**bitwise OR**: 두 정수를 한 비트씩 비교하면서 하나라도 켜져있으면 결과의 비트를 킨다.  
**bitwise XOR**: 두 개중 하나만 켜져있을 때 결과 비트를 킨다.  
**bitwise NOT**: 정수 하나를 입력받아 켜져있는 비트는 끄고, 꺼져 있는 비트는 킨다.  
**shift**: 정수 a의 비트들을 왼쪽 또는 오른쪽으로 원하는 만큼 움직인다. 754를 왼쪽으로 2비트 시프트하면 모든 비트들이 왼쪽으로 두 칸 움직임. 오른쪽 끝 두 개의 비트들은 0으로 채워진다. 오른쪽으로 2비트 시프트하면 모든 비트들이 오른쪽으로 두 칸.. 처음 두 개의 비트들은 0으로 채워짐
  
    c++에서 비트연산자 사용법
    
    연산                                 코드
    ------------------------------------------
    두 정수 a, b를 비트별로 AND 연산     a & b
    두 정수 a, b를 비트별로 OR 연산      a | b
    두 정수 a, b를 비트별로 XOR 연산     a ^ b
    정수 a의 비트별 NOT 연산             ~a
    정수 a를 왼쪽으로 b비트 시프트       a << b
    정수 a를 오른쪽으로 b비트 시프트     a >> b
    
### 유의할 점들
1. 연산자 우선순위: `&, |, ^`등의 비트연산자의 우선순위는 == 혹은 != 등의 비교연산자보다 낮다.   
    
        int c = (6 & 4 == 4);
        위의 식은 4==4가 먼저 계산되고, 이 결과인 1이 6과 비트별 AND연산되어 c = 0이 된다.
        
        int d = ((6 & 4) == 4);
        와 같이 괄호를 쓰는 습관을 들여야 함..
       
2. 64비트 정수를 비트마스크로 사용할 때 오버플로 주의.

        부호없는 64비트 비트마스크 a의 b번 비트가 켜져 있는지 확인하기 위한 코드를
        
        bool isBitSet(unsigned long long a, int b) {
            return (a & (1 << b)) > 0;
        }
        
        와 같이 작성했을 때, 1은 부호 있는 32비트 상수로 취급되기 때문에 b가 32보다 크면 1<<b에서 오버플로 발생.
        1이 부호 없는 64비트임을 알려주도록 1ull 과 같이 접미사 ull을 붙여줘야 한다.
        
3. 부호 있는 정수형.

        부호있는 정수형에서 최상위 비트가 켜진 숫자는 음수를 표현한다. 
        변수의 모든 비트를 다 쓰고 싶을 때는 부호 없는 정수형을 쓰는 것이 좋다.
        
4. N비트 정수를 N비트 이상 왼쪽으로 Shift하는 오류. 환경에 따라 0이 아닌 다른 결과를 낼 수도 있다.        

## 비트마스크를 이용한 집합의 구현
비트마스크를 사용한 가장 중요한 사례는 집합을 구현하는 것.  
N비트 정수 변수는 0부터 N-1까지의 정수 원소를 가질 수 있는 집합이 된다.  
이떄 원소 i가 집합에 속해 있는지 여부는 $2^i$을 나타내는 비트가 켜져 있는지 여부로 나타낸다.  
6개의 원소를 갖는 집합 `{1, 4, 5, 6, 7, 9}`를 표현하는 정수는 $2^1 + 2^4 + 2^5 + 2^6 + 2^7 + 2^9 = 10 1111 0010_2 = 754$가 된다.
  
### 피자집 예제
피자집에 0~19까지 번호를 갖는 스무 개의 토핑이 있다.  
주문시 각 토핑마다 넣을지 안넣을지 고를 수 있다.  
한 피자의 정보는 최대 스무 종류의 원소만을 가지는 집합이 되고 이를 비트마스크를 이용해 표현.  
20개의 boolean 배열을 이용할 수 있겠지만 비트마스크를 이용하면 다양한 비트연산을 이용해 집합연산을 빠르고 간단하게 구현.  

#### 1. 공집합과 꽉 찬 집합 구하기.
토핑을 하나도 올리지 않은 피자(공집합) vs. 모든 토핑을 올린 피자  
**공집합**: 상수 0이 공집합 표현! 매우 간단하다.  
**전체집합**: 20개 비트가 모두 켜진 수. `int fullPizza = (1 << 20) - 1;` 이것도 간단.  
1 << 20 은 이진수로 나타냈을 때 처음 1 뒤에 0이 20개 있는 수. 즉 $2^{21}$. 여기서 1을 빼면 마지막 20개 비트가 모두 1인 수가 된다. 
  
#### 2. 원소 추가하기
집합의 가장 기초적인 연산은 원소를 추가/삭제하는 것이다.  
비트마스크를 사용한 집합에서 원소를 추가하는 것은 해당 비트를 1로 키는 것.  
피자에 페퍼로니 토핑을 추가하고 싶을때, 페퍼로니의 번호가 $p(0<=p<20)$라면,   

    toppings |= (1 << p);
   
와 같은 코드로 추가. 1을 왼쪽으로 p비트 시프트하면 p번 비트만 켜진 상수가 된다.($2^p$)  
따라서 이 값을 원래 집합인 toppings와 bitwise or연산하면 p번째 비트는 반드시 켜지게 된다. 이미 들어가 있으면 변화 없음.  
  
#### 3. 어떤 원소가 집합에 포함돼있는지 확인
토핑 목록에 페퍼로니가 잘 추가돼있는지 확인해보자. 집합 toppings에 페퍼로니(p번째)가 포함돼있는지 다음 코드로 알 수 있다.

    if (toppings & (1 << p)) cout << "pepperoni is in \n";
    
`&`연산의 결과값이 0 또는 1 << p 임에 유의. 논리연산이 아니므로 다음과 같이 작성하면 틀리다.

    if ((toppings & (1 << p)) == 1) cout << ...
    
#### 4. 어떤 원소를 집합에서 삭제.
토핑 목록에서 페퍼로니를 다시 빼기로 한다면 먼저 생각해볼 수 있는 방법은,

    toppings -= (1 << p);
    
하지만 이 코드는 이미 토핑 목록에서 페퍼로니가 빠져있는경우 문제가 된다.  
이미 해당 원소가 없을 때에도 정상적으로 동작하게 하기 위해서는,  

    toppings &= ~(1 << p);
    
와 같이 써야한다.  
C++의 `~`연산자는 비트별 NOT 연산을 수행하므로, `~(1 << p)`는 해당 비트만 꺼지고 나머지는 다 켜진 숫자가 된다.  
이 숫자와 bitwise AND 연산을 수행하면 toppings의 나머지 비트는 유지되고 p번 비트는 항상 꺼짐이 보장된다.   
  
#### 5. 원소의 토글(toggle). 
해당 비트가 켜져있으면 끄고, 꺼져있으면 키는 것.  XOR연산으로 쉽게 구현.  
    
    toppings ^= (1 << p); 
   
#### 6. 두 집합에 대해 연산하기.
p번 토핑을 추가/삭제할 때 `1<<p`를 이용했는데, 이는 크기가 1인 집합으로도 볼 수 있다.  
크기가 더 큰 집합을 사용해도 이 연산들 그대로 적용.  
두 개의 집합 a, b의 합집합과 차집합.  

    int added = (a | b);          // a와 b의 합집합
    int intersection = (a & b);   // a와 b의 교집합
    int removed = (a & ~b);       // a에서 b를 뺀 차집합
    int toggled = (a ^ b);        // a와 b중 하나에만 포함된 원소들의 집합.. 즉 합집합에서 교집합을 뺀 부분.
    
이 코드의 수행 시간은 원소 하나에 대해 수행하는 것과 다르지 않다.  
집합 간 연산을 빠르게 할 수 있다는 점이 비트마스크를 이용한 집합 표현의 큰 장점.    
  
#### 7. 집합의 크기 구하기.
비트마스크를 이용할 때 집합에 포함된 원소의 수를 구하는 쉬운 방법은 딱히 없다.  
각 비트를 순회하면서 켜져 있는 비트의 수를 직접 세야함. 재귀적으로 작성.  

    int bitCount(int x) {
        if (x == 0) return 0;
        return x % 2 + bitCount(x / 2);
    }

최적화에 여러 방법이 있지만, 컴파일러, 언어별로 관련된 내장 명령어 제공.  

    gcc / g++    |    __builtin_popcount(toppings)
    Visual C++   |    __popcnt(toppings)
    
와 같이 쓸 수 있다. 이거 쓰는게 훨씬 빠름.  

#### 8. 최소 원소 찾기.
집합에 포함된 가장 작은 원소를 찾는 것..  
`"이 정수의 이진수 표현에서 끝에 붙어 있는 0이 몇 개인가?"`의 형태로 지원됨..   
켜져있는 최하위 비트의 번호를 반환

    gcc / g++    |    __builtin_ctz(toppings)
    Visual C++   |    __BitScanForward(&index, toppings)
    
직접 구하려면?  

    int firstTopping = (toppings & -toppings);
    
대부분의 컴퓨터가 음수를 표현하기 위해 2의 보수(Two`'`s complement)를 사용한다는 점을 이용.  
음수 -toppings를 표현하기 위해 toppings에 비트별 NOT 연산을 적용하고, 그 결과에 1을 더함.  
  
toppings에서 켜진 최하위 비트가 $2^i$라고 하면, 마지막 i + 1 자리에는 1 뒤에 i개의 0이 있는 형태이다.  
여기에 비트별 NOT 연산을 적용하면 마지막 i + 1 자리는 0 뒤에 i개의 1이 있는 형태가 되고,  
여기에 다시 1을 더하면 다시 1과 i개의 0이 있는 형태가 된다.  
이 때 $2^i$보다 큰 상위비트들에는 NOT연산이 적용된 상태이기 때문에 두 수를 AND하면 항상 최하위 비트만을 얻을 수 있다.  
이 기법은 펜윅 트리에서 유용하게 사용됨.  
  
#### 9. 최소 원소 지우기. 
종종 최소 원소가 무엇인가와 상관없이 최소 원소를 지우는 연산이 유용할 때가 있다.

    toppings &= (toppings - 1);
    
toppings - 1 의 이진수 표현을 보면 켜져있는 최하위 비트를 끄고, 그 밑의 비트들을 전부 켠 것.  
두 값을 비트별 AND연산하면 최하위 비트와 그 이하의 비트들은 전부 0이 된다.  

          40 = 101000 (2)
    AND   39 = 100111 (2)
         -----------------
          32 = 100000 (2)
          
이 방법은 어떤 정수가 2의 거듭제곱 값인지 확인할 때도 유용하게 쓰인다.  
2의 거듭제곱 값들의 이진수 표현에는 켜진 비트가 하나밖에 없기 때문에, 최하위 비트를 지우면 0이 됨.  

#### 10. 모든 부분 집합 순회하기!!!!  (중요!!!!)
    
    for (int subset = pizza; subset; subset = ((subset - 1) & pizza)) {
        // subset은 pizza의 부분집합.
    }
    
다음 부분 집합을 구하는 식 (subset - 1) & pizza에서,  
subset에서 1을 빼면 켜져있던 최하위 비트가 꺼지고, 그 밑의 비트들은 전부 켜지게 된다.  
이 결과와 pizza의 교집합을 구하면 그 중 pizza에 속하지 않는 비트들은 모두 꺼짐.  
이 연산을 반복하면 pizza의 모든 부분 집합을 방문할 수 있다.  
이 코드에서 for문은 subset=0인 시점에 종료하므로 공집합은 방문하지 않는다.  
이를 boolean array를 이용하여 한다면 재귀함수를 이용해야함..  

### 비트마스크 응용 예제
#### 1. 지수 시간 동적 계획법
배열 입력을 갖는 함수를 메모이제이션하는 방법을 다루는 9.11절 참고.  
배열 대신 정수로 집합을 표현하면 곧장 배열의 인덱스로 쓸 수 있어 메모이제이션 구현도 간단해진다.  
  
#### 2. 에라토스테넷의 체
알고리즘은 매우 빠르게 동작. 메모리가 더 문제.  
체를 구현할 때 범위 내의 각 정수가 지워졌는지 여부를 저장해야 하는데,  
만약 boolean배열을 이용하면, 32비트 정수 표현범위를 다루기 위해 4GB의 메모리를 써야함.  
bool 이 1 bytes이기 때문!! 근데 왜 1bytes나 필요하지??  
짝수를 제외해 2GB로 줄일 수는 있지만 여전히 많음.  
비트마스크를 사용하면 이를 다시 1/8로 줄일 수 있다.  
unsigned char을 이용하면 부호를 나타내던 최상위 비트까지 이용해서 수를 나타낼 수 있다.  
MAX_N개의 원소를 갖는 불린 값 배열을 다음과 같은 배열로 대체한다.

    unsigned char sieve[(MAX_N + 7) / 8];
    
이 배열은 MAX_N / 8 바이트만 써서 MAX_N개의 원소를 갖는 불린 값 배열을 구현한다.  
k번 원소가 참인지 알기 위해서는 k/8번째 원소의 k%8번째 비트가 켜져 있는지를 확인하면 된다.  
k >> 3 연산을 통해 해당하는 index에 접근하고 k & 7을 통해 k % 8값을 구한다.

    // 비트마스크를 사용하는 에라토스테네스의 체의 구현
    int n;
    unsigned char sieve[(MAX_N + 7) / 8];
    
    // k가 소수인지 확인
    inline bool isPrime(int k) {
        return sieve[k >> 3] & (1 << (k & 7));
    }
    
    // k가 소수가 아니라고 표시한다.
    inline void setComposite(int k) {
        sieve[k >> 3] &= ~(1 << (k & 7));
    }
    // 비트마스크를 사용하는 에라토스테네스의 체의 구현
    // 이 함수를 수행하고 난 뒤, isPrime()을 이용해 각 수가 소수인지 알 수 있다.
    void eratosthenes() {
        memset(sieve, 255, sizeof(sieve));
        setComposite(0);
        setComposite(1);
        int wqrtn = int(sqrt(n));
        for (int i = 2; i <= sqrtn; i++)
            // 이 수가 아직 지워지지 않았다면
            if (isPrime(i))
                // i의 배수 j들에 대해 isPrime(j) = flase로 둔다.
                // i * i 미만의 배수들은 이미 지워졌으므로 신경쓰지 않는다.
                for (int j = i * i; j <= n; j += i)
                    setComposite(j);
    }

https://m.blog.naver.com/PostView.nhn?blogId=occidere&logNo=220973318428&proxyReferer=https%3A%2F%2Fwww.google.com%2F 참고
  
#### 3. 15퍼즐 상태 표현하기
비트마스크를 이용하여 꼭 불린 값 배열만 표현해야 하는 것은 아니다.  
표현해야 하는 값의 범위가 작을 때는 2비트씩, 3비트씩 묶어서 배열로 쓸 수 있다.  
15퍼즐의 상태는 0부터 15까지의 숫자가 들어있는 4x4 크기의 배열로 표현.  
각 숫자는 4비트로 표현, 16개의 숫자가 있기 때문에 비트마스크를 사용하면 이 배열 전체를 64비트 정수 하나로 표현.  
크기 16인 char배열에 비해 크기가 절반으로 줄은 데다가 64비트 아키텍처의 경우 상태 전체를 워드 하나에 넣을 수 있다.  

    // 부호없는 64비트 정수를 배열로 다루기 위한 함수 구현
    typedef unsigned long long uint64;
    // mask의 index 위치에 쓰인 값을 반환한다.
    int get(uint64 mask, int index) {
        return (mask >> (index << 2)) & 15;
    }
    // mask의 index위치를 value로 바꾼 결과를 반환.
    uint64 set(uint64 mask, int index, uint64 value) {
        return mask & ~(15LL << (index << 2)) | (value << (index << 2));
    }

#### 4. O(1) 우선순위 큐
우선순위 큐에 자료를 추가하거나 삭제하는 작업은 O(logN) 소요.  
우선순위가 특정 범위로 제한되어 있을 경우 비트마스크를 이용하면 모든 작업을 O(1)에 할 수 있는 우선순위 큐를 만들 수 있다.  
우선순위가 1이상 140이하의 정수라 하면,  
각 우선순위를 갖는 원소들을 담는 140개의 큐를 만들고, 각 큐에 원소가 있는지 여부를 비트마스크로 표현.  
140개 불린 값을 64비트 정수 3개에 저장하면 첫 번째 비트를 찾는 연산을 이용해 모든 큐를 뒤질 필요 없이  
가장 우선순위가 높은 원소가 어디에 있는지를 쉽게 찾을 수 있다.  
  
#### 5. 예제: 극대 안정 집합
N(N<=20)개의 화학 물질을 운반하는데, 같이 두었을 때 반응해 폭발하는 물질들이 있다.  
이 때 한 상자에 넣어도 폭발하지 않는 물질의 집합을 안정적이라고 부르자.  
물질이 하나만 있으면 항상 안정적이다.  
안정적인 여러 집합들 중 물질을 하나라도 추가하면 폭발이 일어나는 집합들을 극대 안정 집합(maximal stable set)이라고 부른다.  
이 또한 여러 개가 있을 수 있다.  
각 화학 물질의 정보가 주어질 때 극대 안정 집합의 수를 세는 방법?  
1. 그래프 탐색 알고리즘 이용 모든 안정 집합을 만들어가기. 더이상 물질을 추가할 수 없는 집합의 수를 반환.  
2. 비트마스크 이용. 조금 비효율적인 하지만 입력의 크기가 작을때 매우 짧게 구현 가능.  
  
먼저 어떤 집합이 안정적인지를 판단하는 코드에 비트마스크를 적용할 수 있다.  
기본적으로 이중 for문이 필요하지만 비트마스크에 저장해두면 for문 하나로 가능.  

    // 비트마스크를 이용해 모든 극대 안정 집합의 수를 세는 함수 구현
    int n;
    // explodes[i]는 i와 같이 두었을 때 폭발하는 물질 집합의 비트마스크 표현
    int explodes[MAXN];
    // 주어진 집합이 안정적인지 확인한다.
    bool isStable(int set) {
        for (int i = 0; i < n; i++)
            // 집합에 포함된 i번째 원소와 같이 두었을 때 폭발하는 물질이 set에 있다면
            if ((set & (1 << i)) && (set & explodes[i]))
                return false;
        return true;
    }
    // 모든 극대 안정 집합의 수를 센다.
    int countStableSet() {
        int ret = 0;
        // 모든 집합을 만들어 보자.
        for (int set = 1; set < (1 << n); set++) {
            // 안정적이 아니라면 셀 필요 없다.
            if (!isStable(set)) continue;
            // 극대 안정 집합인지 확인하기 위해, 넣을 수 있는 다른 물질이 있나 확인한다.
            bool canExtend = false;
            for (int add = 0; add < n; add++) 
                // add가 집합에 포함되어 있지 않고, set에 add를 넣어도 안정적이라면
                if ((set & (1 << add)) == 0 && (explodes[add] & set) == 0) {
                    canExtend = true;
                    break;
                }
            if (!canExtend)
                ret++;
        }
        return ret;
    }
    
$2^N$개의 모든 부분집합을 전부 만들어 보면서 각 집합이 안정적인지, 안정적이라면 다른 물질을 추가할 수 있는지 확인할 수 있다.  
어떤 안정 집합에 다른 원소 add를 넣을 수 있는지를 isStable()를 다시 호출하는 대신 explodes[add] & set이 0인지만 확인한다.               
            


### 문제: 졸업 학기
졸업 전에 전공 과목 N개 중 K개 이상을 수강해야 하는데, 각 과목은 선수과목을 미리 수강해야하고,  
각 학기마다 모든 과목이 개설되는 것이 아니다. 최소 학기에 졸업을 하려면?  
각 과목의 정보와 앞으로 M학기 동안 개설될 과목의 목록이 주어질 때, 최소 몇 학기를 다녀야 졸업할 수 있는지 계산하는 프로그램 작성.  
한 과목도 수강하지 않는 학기는 휴학한 것으로 하며 다닌 학기 수에 포함되지 않는다.  
  
#### 입력
첫줄에 테스트 케이스의 수 C(C<=50)
각 테스트케이스 첫 줄에 전공 과목의 수 N(1<=N<=12), 들어야 할 과목의 수 K(0<=K<=N), 학기의 수 M(1<=M<=10),  
한 학기에 최대로 들을 수 있는 과목의 수 L(1<=L<=10)이 주어진다. 각 과목은 0~N-1로 번호.  
이 후 N줄에 0번 과목부터 각 과목의 정보가 주어짐.  
선수 과목의 수 $R_i(0<=R_i<=N-1)$가 처음 주어지고, 그 후 $R_i$개의 정수로 선수 과목의 번호가 주어짐.  
이 후 M줄에는 이번학기부터 순서대로 각 학기의 정보가 주어짐.  
각 줄에는 해당 학기에 개설되는 과목의 숫자 $C_i(1<=C_i<=10)$가 주어지고, 그 후 $C_i$개의 정수로 개설되는 과목의 번호들이 주어짐.   
  
#### 출력
각 테스트 케이스마다 한 줄에 다녀야 할 최소 학기 수 출력. M학기 내 졸업이 불가능하면 IMPOSSIBLE 출력.  

9.11절 참고. 전형적인 지수 시간 동적 계획법.  
우선 완전탐색알고리즘을 설계하고 메모이제이션으로 최적화.  
완전탐색으로 풀기 위해서는 문제를 여러 조각으로 나눈 다음 한 조각을 해결하고 나머지를 재귀 호출을 통해 해결.  
이 문제를 풀기 위해서 각 학기를 한 조각으로 쪼개보자.  
다음과 같이 부분 문제를 정의해보자.  
`graduate(semester, taken) = 현재 학기가 semester이고, 지금까지 들은 과목의 집합이 taken일떄, 앞으로 다녀야 하는 최소 학기의 수.`  
graduate()를 완전 탐색으로 구현하는 방법은 각 학기마다 들을 수 있는 모든 과목의 조합들을 하나하나 시도해 보는 것.  
이번 학기에 개설되면서 선수 과목을 모두 들은 과목들 중, L개 이하의 모든 조합을 시도하는 것.  
이를 위한 일반적인 방법은 재귀호출이지만, 이미 graduate()라는 재귀 함수를 사용중. 또 재귀호출하는 choose()라는 함수를 만들면 복잡..  
이 때 비트마스크가 유용하다.  
  
어떤 과목의 선수과목을 전부 들었는지 확인: taken과 prerequisite[i]의 교집합이 prerequisite[i]와 같은지 확인.  
canTake에서 아직 선수 과목을 다 듣지 않아 들을 수 없는 과목들을 미리 걸러냄.  
이번 학기에 들을 수 있는 과목만이 canTake에 남기 때문에 이 집합의 부분 집합을 순회하면 된다.  
이미 들은 과목의 수나 이번 학기에 들을 과목의 수를 세기 위해 비트의 수를 세는 함수 bitCount() 사용.  

    // 구현
    const int INF = 987654321;
    int n, k, m, l;
    // prerequisite[i] = i번째 과목의 선수과목의 집합
    int prerequisite[MAXN];
    // classes[i] = i번째 학기에 개설되는 과목의 집합
    int classes[10];
    int cache[10][1<<MAXN];
    // n의 이진수 표현에서 켜진 비트의 수를 반환
    int bitCount(int n);
    // 이번 학기가 semester이고, 지금까지 들은 과목의 집합이 taken일 때
    // k개 이상의 과목을 모두 들으려면 몇 학기나 더 있어야 하는가?
    // 불가능한 경우 INF 반환
    int graduate(int semester, int taken) {
        // 기저 사례: k개 이상의 과목을 이미 들은 경우
        if (bitCount(taken) >= k) return 0;
        // 기저 사례: m학기가 전부 지난 경우
        if (semester == m) return INF;
        // 메모이제이션
        int& ret = cache[semester][taken];
        if (ret != -1) return ret;
        ret = INF;
        // 이번 학기에 들을 수 있는 과목 중 아직 듣지 않은 과목들을 찾는다.
        int canTake = (classes[semester] & ~taken);
        // 선수 과목을 다 듣지 않은 과목들 걸러냄.
        for (int i = 0; i < n; i++)
            if ((canTake & (1 << i) && (taken & prerequisite[i]) != prerequisite[i])
                canTake &= ~(1 << i);
        // 이 집합의 모든 부분집합을 순회한다.
        for (int take = canTake; take > 0; take = ((take - 1) & canTake)) {
            // 한 학기에 l과목까지만 들을 수 있다.
            if (bitCount(take) > l) continue;
            ret = min(ret, graduate(semester + 1, taken | take) + 1);
        }
        // 이번 학기에 아무것도 듣지 않을 경우
        ret = min(ret, graduate(semester + 1, taken));
        return ret
    }
    
#### 시간복잡도
graduate() 내부의 반복문의 수행횟수가 두 인자에 모두 영향을 받기 때문에 계산이 까다롭다.  
상한만 생각한다면 canTake의 모든 부분 집합을 순회할 때 최대 $2^C$의 시간이 걸리고,  
전체 $M*2^{N+C}$개의 부분 문제가 있으므로 프로그램의 전체 시간 복잡도는 $O(M*2^{N+C})$가 된다.  